In [20]:
"""
Блок 2. Партицирование
"""

import libs.db_connection

# Интересует конкретный пользователь
sql1 = "explain (analyze) select * from block11 where category < 10 and type = 'click'"
            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.43..57198.68 rows=3766 width=17) (actual time=0.373..169.505 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Index Scan using block1_type2 on block11  (cost=0.43..55822.08 rows=1569 width=17) (actual time=0.173..129.942 rows=1504 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Index Cond: ((type)::text = 'click'::text)")]),
 RealDictRow([('QUERY PLAN', '        Filter: (category < 10)')]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 165285')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.524 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 169.850 ms')])]

In [42]:
# Создадим партицированную таблицу и определим партиции
sql1 = """create table block11_partitioned (event_id int, user_id int, category int, type varchar) PARTITION BY LIST(type);
CREATE TABLE block11_click PARTITION OF block11_partitioned FOR VALUES IN ('click');
CREATE TABLE block11_show PARTITION OF block11_partitioned FOR VALUES IN ('show');
"""

libs.db_connection.execute(sql1)

In [43]:
# Загрузим данные
sql1 = "INSERT INTO block11_partitioned SELECT * from block11;"
            
libs.db_connection.execute(sql1)

In [51]:
# Проверяем наш запрос
sql1 = "explain (analyze) select * from block11_partitioned where category < 10 and type = 'click'"
            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..7784.69 rows=4694 width=18) (actual time=0.189..20.170 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on block11_click block11_partitioned  (cost=0.00..6315.29 rows=1956 width=18) (actual time=0.012..14.863 rows=1504 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: ((category < 10) AND ((type)::text = 'click'::text))")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 165285')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.274 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 20.413 ms')])]

In [53]:
# Можно работать напрямую с партицией
sql1 = "explain (analyze) select * from block11_click where category < 10 and type = 'click'"
            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..7263.48 rows=4694 width=18) (actual time=0.312..21.893 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on block11_click  (cost=0.00..5794.08 rows=1956 width=18) (actual time=0.018..17.372 rows=1504 loops=3)')]),
 RealDictRow([('QUERY PLAN', '        Filter: (category < 10)')]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 165285')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.180 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 22.138 ms')])]

In [85]:
# Задание: Для ускорения инкрементальной загрузки создать партиции на даты:
# 2022-05-20
# 2022-05-21
# 2022-05-22
# 2022-05-23

sql = "explain (analyze) select * from task11 where dt = '2022-05-20 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])


[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..232554.54 rows=1 width=21) (actual time=547.157..549.878 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on task11  (cost=0.00..231554.44 rows=1 width=21) (actual time=530.543..530.543 rows=0 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: (dt = '2022-05-20 00:00:00+00'::timestamp with time zone)")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 6666667')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.120 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 1.017 ms, Inlining 0.000 ms

In [78]:
# Создаем таблицу и партиции
sql1 = """create table task11_partitioned (event_id int, user_id int, type varchar, dt timestamptz) PARTITION BY RANGE (dt);
create table task11_2022_05_19 partition of task11_partitioned for values from ('2022-05-19') to ('2022-05-20');
create table task11_2022_05_20 partition of task11_partitioned for values from ('2022-05-20') to ('2022-05-21');
create table task11_2022_05_21 partition of task11_partitioned for values from ('2022-05-21') to ('2022-05-22');
create table task11_2022_05_22 partition of task11_partitioned for values from ('2022-05-22') to ('2022-05-23');
create table task11_2022_05_23 partition of task11_partitioned for values from ('2022-05-23') to ('2022-05-24');
"""
libs.db_connection.execute(sql1)

In [79]:
# Загружаем данные
sql1 = """insert into task11_partitioned select * from task11;"""

libs.db_connection.execute(sql1)

In [86]:
# Проверяем производительность

sql = "explain (analyze) select * from task11_partitioned where dt = '2022-05-20 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..78201.52 rows=1 width=21) (actual time=176.729..178.946 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on task11_2022_05_20 task11_partitioned  (cost=0.00..77201.42 rows=1 width=21) (actual time=174.361..174.362 rows=0 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: (dt = '2022-05-20 00:00:00+00'::timestamp with time zone)")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 2222688')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.382 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 179.019 ms')])]

In [88]:
# Сравниваем производительность с индексом

sql = "explain (analyze) select * from task1 where dt = '2022-05-20 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..116779.03 rows=1 width=21) (actual time=304.026..306.619 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on task1  (cost=0.00..115778.93 rows=1 width=21) (actual time=283.989..283.990 rows=0 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: (dt = '2022-05-20 00:00:00+00'::timestamp with time zone)")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 3333333')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.127 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 0.958 ms, Inlining 0.000 ms,